In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc

from datetime import datetime
import requests
import json
from pandas import json_normalize

from tqdm import tqdm
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston

In [2]:
today = datetime.today().strftime('%Y%m%d')
year_list =[2020, 2021, 2022]
df_ho =pd.DataFrame([])
for i in year_list:
    today_year = i

    key = 'UhnjyvmWEq5SH4yCqPylCPQSp4NdA524up%2FM1CrsUf3143Vmj3Rwx7PGHxq5YZY4mFHNHQ%2F0ixGwapRFApPsaw%3D%3D'
    url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear=' + str(today_year) + '&ServiceKey=' + str(key)
    response = requests.get(url)
    if response.status_code == 200:
        json_ob = json.loads(response.text)
        holidays_data = json_ob['response']['body']['items']['item']
        dataframe = json_normalize(holidays_data)
    df_ho = pd.concat([df_ho, dataframe])

In [3]:
df_ho=df_ho.reset_index()

In [4]:
df_ho['locdate']=df_ho['locdate'].reset_index(drop=True)

In [5]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [6]:
csv_to_parquet('C:/Users/82108/JejuRoad/JejuRoadData/open/train.csv', 'train')
csv_to_parquet('C:/Users/82108/JejuRoad/JejuRoadData/open/test.csv', 'test')

train Done.
test Done.


In [7]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

In [8]:
str_col = ['start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [9]:
t_train = train.copy()
t_test = test.copy()

In [10]:
y = t_train['target'] 

x = t_train.drop(['id','target' , 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code'], axis=1)

t_test = t_test.drop(['id' , 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code'], axis=1)

print(x.shape)
print(y.shape)
print(t_test.shape)

(4701217, 13)
(4701217,)
(291241, 13)


In [12]:
col_list = ['base_hour','lane_count','day_of_week','maximum_speed_limit']
for i in col_list:
    x = pd.get_dummies(x,columns = [i])
    t_test = pd.get_dummies(t_test,columns = [i])

In [13]:
A_cnt = x['road_name'].value_counts()
print("road_name :\n", x['road_name'].value_counts())
print()

road_name :
 일반국도12호선    1046092
-            569463
일반국도16호선     554510
일반국도95호선     248181
일반국도11호선     215701
             ...   
애원로            7718
아봉로            7342
남조로            6813
호서중앙로          2819
호근로             587
Name: road_name, Length: 61, dtype: int64



In [14]:
A_cnt = x['start_node_name'].value_counts()
print("start_node_name :\n", x['start_node_name'].value_counts())
print()

start_node_name :
 고성교차로              42169
산신주유소              29386
미수2교               26635
영주교                24381
봉개교 시종점            24290
                   ...  
외산리사무소               934
와산리                  881
성미가든                 788
서귀포시 호근동 1838-4      744
CU서귀호근로점             744
Name: start_node_name, Length: 487, dtype: int64



In [15]:
for i in (df_ho['locdate']):
    x.loc[(x['base_date'] == i),'base_date'] = 1
x.loc[(x['base_date'] > 1),'base_date'] = 0

for i in (df_ho['locdate']):
    t_test.loc[(t_test['base_date'] == i),'base_date'] = 1
    
t_test.loc[(t_test['base_date'] > 1),'base_date'] = 0    

In [16]:
from sklearn.preprocessing import OneHotEncoder

##### A_cnt = x['start_longitude'].value_counts()
print("road_name :\n", x['start_longitude'].value_counts())
print()

##### x_scaler=x.copy()

##### x_scaler=x_scaler[['start_latitude', 'start_longitude','end_latitude', 'end_longitude']]

##### from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_scaler = scaler.fit_transform(x_scaler)

pd.DataFrame(x_scaler, columns = ['start_latitude', 'start_longitude','end_latitude', 'end_longitude'])

In [58]:
x_sample = x_test.iloc[:100,:]

In [54]:
y

0          52.0
1          30.0
2          61.0
3          20.0
4          38.0
           ... 
4701212    20.0
4701213    65.0
4701214    30.0
4701215    73.0
4701216    35.0
Name: target, Length: 4701217, dtype: float64

In [55]:
y_sample = y.iloc[:100]

In [56]:
y_sample

0     52.0
1     30.0
2     61.0
3     20.0
4     38.0
      ... 
95    23.0
96    39.0
97    21.0
98    48.0
99    19.0
Name: target, Length: 100, dtype: float64

In [59]:
x_sample

,base_date,start_turn_restricted,end_turn_restricted,base_hour_0,base_hour_1,base_hour_2,base_hour_3,base_hour_4,base_hour_5,base_hour_6,...,road_name_지방도1119호선,road_name_지방도1120호선,road_name_지방도1132호선,road_name_지방도1136호선,road_name_지방도97호선,road_name_첨단로,road_name_태평로,road_name_한천로,road_name_호근로,road_name_호서중앙로
1668774,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
299795,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2245633,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
434484,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4562091,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730384,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
877394,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
848681,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2297881,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### x_sample['start_latitude']

##### x_sample['x1'] = 10000*(round((x_sample['start_latitude'] * x_sample['start_longitude']),4)-4200)
x_sample['x1']= x_sample['x1'].astype('int')
ohe = OneHotEncoder()
sample_latitude = ohe.fit_transform(x[['x1']])

##### sample_cnt = x_sample['x1'].value_counts()
print("road_name :\n", x_sample['x1'].value_counts())
print()

##### x_sample['x1']= x_sample['x1'].astype('object')

##### x_sample['x1'].dtypes

##### ass = (col for col in ohe.categories_[0])

##### for col in ohe.categories_[0]:
ssa=[]
    print(str(col))
    ssa.append(str(col))

##### x_sample = pd.get_dummies(x_sample,columns = ['x1'])

##### sample_locate = pd.DataFrame(sample_latitude, columns=['fcol_{}'.format(ass)])

In [17]:
x['x1'] = 10000*(round((x['start_latitude'] * x['start_longitude']),4)-4200)

In [18]:
x['x1'].dtypes

dtype('float64')

In [19]:
x['x1']= x['x1'].astype('int')

In [20]:
A_cnt = x['x1'].value_counts()
print("road_name :\n", x['x1'].value_counts())
print()

road_name :
 82052     23291
375457    20775
319720    18137
405618    18084
379926    18076
          ...  
428081      881
357354      788
77807       744
83631       744
79174       587
Name: x1, Length: 586, dtype: int64



In [21]:
x = pd.get_dummies(x,columns = ['x1'])

In [22]:
x = pd.get_dummies(x,columns = ['road_name'])

In [23]:
t_test = pd.get_dummies(t_test,columns = ['road_name'])

In [24]:
t_test['x1'] = 10000*(round((t_test['start_latitude'] * t_test['start_longitude']),4)-4200)
t_test['x1']= t_test['x1'].astype('int')
t_test = pd.get_dummies(t_test,columns = ['x1'])

In [27]:
t_test

,base_date,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted,base_hour_0,base_hour_1,...,x1_427538,x1_430639,x1_430754,x1_431684,x1_432033,x1_432102,x1_432816,x1_433829,x1_444920,x1_448681
0,0,산지2교,33.499427,126.541298,0,33.500772,126.543837,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,중문입구,33.258507,126.427003,0,33.258119,126.415840,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,도순3교,33.258960,126.476508,0,33.259206,126.474687,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,아라주공아파트,33.473494,126.545647,0,33.471061,126.545467,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,부록교 시종점,33.501477,126.569223,0,33.496863,126.581230,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291236,0,월산마을,33.472819,126.463030,0,33.470483,126.460846,0,0,0,...,0,0,0,0,0,0,0,0,0,0
291237,0,영주교,33.305359,126.598914,0,33.300796,126.600332,0,0,0,...,0,0,0,0,0,0,0,0,0,0
291238,0,마리나사거리,33.493624,126.496769,1,33.497500,126.496946,1,0,0,...,0,0,0,0,0,0,0,0,0,0
291239,0,국민의료보험관리공단,33.254782,126.507014,0,33.255659,126.507333,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
temp1=[]
for i in (x.columns):
    temp1.append(i)
    print('"{}"'.format(i),end=",")

"base_date","start_node_name","start_latitude","start_longitude","start_turn_restricted","end_latitude","end_longitude","end_turn_restricted","base_hour_0","base_hour_1","base_hour_2","base_hour_3","base_hour_4","base_hour_5","base_hour_6","base_hour_7","base_hour_8","base_hour_9","base_hour_10","base_hour_11","base_hour_12","base_hour_13","base_hour_14","base_hour_15","base_hour_16","base_hour_17","base_hour_18","base_hour_19","base_hour_20","base_hour_21","base_hour_22","base_hour_23","lane_count_1","lane_count_2","lane_count_3","day_of_week_금","day_of_week_목","day_of_week_수","day_of_week_월","day_of_week_일","day_of_week_토","day_of_week_화","maximum_speed_limit_30.0","maximum_speed_limit_40.0","maximum_speed_limit_50.0","maximum_speed_limit_60.0","maximum_speed_limit_70.0","maximum_speed_limit_80.0","x1_-28850","x1_-25626","x1_-3221","x1_-1710","x1_3912","x1_4233","x1_4408","x1_10808","x1_14575","x1_22438","x1_25999","x1_27020","x1_27876","x1_28631","x1_28855","x1_30061","x1_32445","x1

In [26]:
temp2=[]
for i in (t_test.columns):
    temp2.append(i)
    #print('"{}"'.format(i),end=",")

In [27]:
s = set(temp2)
temp3 = [x for x in temp1 if x not in s]

In [28]:
for i in temp3:
    t_test[i] = 0

C:\Users\82108\AppData\Local\Temp\ipykernel_20300\3108360276.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  t_test[i] = 0
C:\Users\82108\AppData\Local\Temp\ipykernel_20300\3108360276.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  t_test[i] = 0
C:\Users\82108\AppData\Local\Temp\ipykernel_20300\3108360276.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inste

In [33]:
t_test

,base_date,start_turn_restricted,end_turn_restricted,base_hour_0,base_hour_1,base_hour_2,base_hour_3,base_hour_4,base_hour_5,base_hour_6,...,road_name_지방도1119호선,road_name_지방도1120호선,road_name_지방도1132호선,road_name_지방도1136호선,road_name_지방도97호선,road_name_첨단로,road_name_태평로,road_name_한천로,road_name_호근로,road_name_호서중앙로
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291236,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
291237,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
291238,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
291239,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
t_test = t_test[["base_date","start_turn_restricted","end_turn_restricted","base_hour_0","base_hour_1","base_hour_2","base_hour_3","base_hour_4","base_hour_5","base_hour_6","base_hour_7","base_hour_8","base_hour_9","base_hour_10","base_hour_11","base_hour_12","base_hour_13","base_hour_14","base_hour_15","base_hour_16","base_hour_17","base_hour_18","base_hour_19","base_hour_20","base_hour_21","base_hour_22","base_hour_23","lane_count_1","lane_count_2","lane_count_3","day_of_week_금","day_of_week_목","day_of_week_수","day_of_week_월","day_of_week_일","day_of_week_토","day_of_week_화","maximum_speed_limit_30.0","maximum_speed_limit_40.0","maximum_speed_limit_50.0","maximum_speed_limit_60.0","maximum_speed_limit_70.0","maximum_speed_limit_80.0","x1_-28850","x1_-25626","x1_-3221","x1_-1710","x1_3912","x1_4233","x1_4408","x1_10808","x1_14575","x1_22438","x1_25999","x1_27020","x1_27876","x1_28631","x1_28855","x1_30061","x1_32445","x1_33311","x1_34481","x1_37532","x1_37610","x1_37749","x1_38015","x1_38095","x1_38685","x1_39326","x1_39417","x1_40227","x1_40518","x1_40690","x1_40780","x1_41067","x1_41163","x1_41266","x1_41487","x1_43531","x1_45257","x1_45621","x1_46790","x1_47043","x1_47542","x1_47734","x1_47975","x1_49943","x1_50001","x1_52755","x1_52941","x1_52955","x1_53362","x1_53398","x1_53414","x1_54389","x1_54987","x1_55082","x1_55405","x1_58442","x1_58625","x1_59311","x1_60246","x1_60438","x1_63068","x1_63281","x1_63301","x1_63339","x1_63967","x1_64327","x1_64476","x1_64771","x1_64832","x1_65334","x1_65547","x1_66030","x1_66049","x1_66058","x1_66068","x1_66087","x1_66750","x1_67263","x1_67299","x1_67335","x1_67353","x1_67857","x1_68005","x1_68409","x1_68460","x1_68512","x1_69632","x1_69660","x1_69687","x1_69911","x1_70119","x1_70847","x1_70868","x1_70878","x1_70888","x1_70892","x1_70909","x1_70914","x1_70973","x1_71001","x1_71760","x1_72259","x1_72806","x1_72842","x1_73076","x1_73168","x1_73716","x1_75057","x1_76174","x1_76229","x1_76809","x1_77276","x1_77587","x1_77658","x1_77807","x1_78635","x1_78742","x1_78757","x1_78816","x1_79174","x1_79488","x1_80014","x1_80766","x1_81288","x1_81315","x1_81634","x1_81643","x1_82052","x1_82565","x1_83631","x1_84377","x1_85006","x1_85599","x1_86161","x1_86350","x1_87070","x1_87731","x1_88248","x1_88539","x1_88757","x1_89236","x1_89611","x1_91082","x1_91257","x1_91373","x1_92151","x1_92174","x1_93482","x1_94975","x1_95519","x1_96157","x1_97078","x1_97174","x1_98774","x1_100268","x1_101193","x1_101286","x1_101683","x1_102283","x1_102436","x1_103806","x1_103904","x1_104147","x1_104346","x1_106737","x1_107016","x1_107481","x1_108068","x1_108337","x1_114886","x1_114966","x1_115502","x1_115727","x1_117691","x1_118158","x1_118380","x1_119426","x1_119711","x1_121069","x1_121224","x1_122148","x1_122727","x1_123496","x1_123658","x1_124796","x1_129196","x1_132673","x1_132943","x1_136383","x1_140320","x1_141208","x1_143212","x1_143621","x1_145510","x1_150320","x1_150654","x1_151475","x1_152673","x1_158918","x1_160937","x1_161516","x1_162373","x1_163307","x1_164221","x1_164251","x1_164567","x1_164921","x1_166392","x1_167790","x1_168786","x1_170362","x1_174835","x1_176067","x1_177165","x1_177619","x1_177773","x1_178824","x1_179521","x1_179834","x1_179885","x1_180111","x1_180442","x1_182439","x1_182497","x1_183908","x1_184152","x1_186271","x1_186711","x1_186746","x1_187237","x1_187357","x1_187420","x1_187852","x1_187857","x1_188909","x1_188982","x1_191048","x1_191773","x1_191871","x1_191911","x1_193859","x1_194880","x1_196769","x1_199044","x1_199311","x1_199780","x1_200735","x1_201237","x1_201893","x1_204857","x1_205349","x1_205597","x1_206239","x1_206343","x1_207245","x1_207463","x1_207713","x1_208235","x1_208290","x1_211464","x1_215020","x1_215276","x1_219333","x1_219602","x1_223460","x1_223720","x1_223986","x1_224311","x1_231207","x1_231324","x1_231346","x1_232795","x1_233175","x1_233738","x1_234396","x1_234687","x1_235183","x1_235411","x1_236004","x1_236890","x1_237188","x1_238832","x1_241271","x1_241454","x1_241597","x1_242875","x1_243509","x1_244192","x1_246707","x1_247358","x1_251616","x1_253166","x1_253617","x1_253935","x1_262142","x1_263410","x1_263563","x1_263800","x1_265428","x1_267147","x1_267226","x1_268233","x1_268288","x1_269638","x1_271639","x1_271661","x1_271814","x1_272055","x1_272245","x1_272727","x1_273450","x1_273888","x1_275721","x1_275964","x1_277007","x1_277318","x1_278050","x1_278306","x1_278554","x1_279317","x1_279350","x1_279871","x1_280144","x1_281773","x1_282968","x1_283346","x1_285154","x1_285622","x1_287956","x1_288450","x1_291004","x1_291586","x1_291633","x1_294241","x1_295771","x1_295995","x1_297421","x1_298473","x1_298641","x1_298702","x1_298859","x1_304165","x1_305927","x1_307894","x1_308042","x1_309011","x1_309311","x1_309606","x1_311787","x1_311893","x1_312367","x1_313420","x1_314907","x1_315131","x1_315770","x1_317403","x1_319131","x1_319269","x1_319438","x1_319720","x1_319839","x1_319870","x1_320006","x1_320308","x1_321198","x1_324449","x1_325052","x1_325208","x1_325468","x1_327057","x1_328284","x1_328438","x1_328955","x1_329022","x1_330740","x1_331693","x1_335679","x1_335820","x1_336211","x1_336243","x1_337611","x1_337641","x1_339309","x1_340366","x1_340457","x1_340564","x1_341084","x1_341163","x1_341577","x1_341947","x1_343355","x1_343756","x1_344229","x1_344391","x1_344960","x1_345120","x1_345280","x1_345561","x1_345637","x1_347304","x1_348168","x1_348432","x1_348514","x1_348914","x1_348958","x1_349453","x1_350645","x1_350949","x1_351055","x1_352942","x1_353188","x1_353320","x1_353555","x1_353719","x1_354277","x1_354458","x1_354566","x1_355003","x1_355207","x1_355523","x1_356109","x1_356288","x1_356293","x1_356579","x1_356814","x1_356854","x1_357296","x1_357354","x1_358099","x1_358224","x1_358227","x1_358744","x1_359062","x1_359250","x1_359344","x1_359530","x1_360394","x1_360447","x1_361476","x1_361752","x1_362388","x1_362748","x1_363263","x1_363325","x1_363573","x1_364349","x1_364537","x1_364894","x1_367031","x1_368316","x1_368352","x1_369210","x1_369916","x1_370457","x1_371350","x1_373314","x1_373375","x1_374326","x1_374839","x1_375388","x1_375457","x1_375887","x1_376747","x1_376850","x1_377073","x1_377560","x1_377689","x1_377727","x1_378280","x1_379326","x1_379399","x1_379926","x1_380209","x1_380925","x1_381702","x1_381805","x1_381885","x1_381935","x1_382147","x1_382240","x1_382723","x1_382983","x1_383964","x1_384931","x1_385874","x1_387659","x1_388175","x1_389467","x1_390327","x1_390609","x1_392139","x1_392377","x1_392757","x1_393163","x1_393276","x1_394279","x1_394299","x1_394390","x1_394844","x1_395079","x1_395897","x1_397512","x1_397957","x1_399329","x1_399994","x1_400354","x1_400648","x1_400742","x1_402560","x1_402649","x1_404264","x1_405618","x1_407074","x1_407718","x1_408086","x1_408190","x1_408341","x1_408558","x1_408854","x1_408904","x1_412420","x1_412633","x1_413100","x1_414441","x1_416539","x1_423207","x1_424600","x1_426415","x1_427372","x1_427538","x1_428081","x1_430639","x1_430754","x1_431684","x1_432033","x1_432102","x1_432816","x1_433410","x1_433829","x1_441135","x1_441914","x1_444053","x1_444920","x1_445199","x1_446649","x1_448681","x1_449692","x1_450834","x1_451858","x1_452258","x1_452493","x1_454359","x1_454726","x1_455271","x1_455300","x1_461046","x1_465506","x1_469295","x1_470370","x1_471442","x1_473303","x1_474836","x1_477335","x1_477788","x1_480047","x1_487971","x1_499579","x1_500718","x1_503371","x1_505257","x1_517565","x1_521535","x1_521665","x1_525640","x1_526179","x1_526894","x1_527440","x1_528288","x1_528870","x1_533137","x1_533212","x1_535596","x1_536511","x1_537265","x1_537784","x1_540803","road_name_-","road_name_경찰로","road_name_고평교","road_name_관광단지1로","road_name_관광단지2로","road_name_관광단지로","road_name_관덕로","road_name_권학로","road_name_남조로","road_name_동문로","road_name_동부관광도로","road_name_동홍로","road_name_번영로","road_name_산서로","road_name_삼무로","road_name_삼봉로","road_name_삼성로","road_name_새서귀로","road_name_서사로","road_name_수영장길","road_name_시민광장로","road_name_시청로","road_name_신광로","road_name_신대로","road_name_신산로","road_name_아봉로","road_name_애원로","road_name_애조로","road_name_어시천교","road_name_연동로","road_name_연북2교","road_name_연북로","road_name_연삼로","road_name_외도천교","road_name_일반국도11호선","road_name_일반국도12호선","road_name_일반국도16호선","road_name_일반국도95호선","road_name_일반국도99호선","road_name_일주동로","road_name_임항로","road_name_제2거로교","road_name_중문로","road_name_중산간서로","road_name_중앙로","road_name_중정로","road_name_지방도1112호선","road_name_지방도1115호선","road_name_지방도1116호선","road_name_지방도1117호선","road_name_지방도1118호선","road_name_지방도1119호선","road_name_지방도1120호선","road_name_지방도1132호선","road_name_지방도1136호선","road_name_지방도97호선","road_name_첨단로","road_name_태평로","road_name_한천로","road_name_호근로","road_name_호서중앙로"]]

##### t_test= t_test.drop(['start_latitude','start_longitude','end_latitude','end_longitude'],axis=1)

In [35]:
x= x.drop(["start_node_name",'start_latitude','start_longitude','end_latitude','end_longitude'],axis=1)

In [ ]:
x_test = x.copy()

##### x_test= x_test[:].astype('int')

##### x=x.drop(["x1_267147","x1_267226","x1_268233","x1_268288","x1_269638","x1_271639","x1_271661","x1_271814","x1_272055","x1_272245","x1_272727","x1_273450","x1_273888","x1_275721","x1_275964","x1_277007","x1_277318","x1_278050","x1_278306","x1_278554","x1_279317","x1_279350","x1_279871","x1_280144","x1_281773","x1_282968","x1_283346","x1_285154","x1_285622","x1_287956","x1_288450","x1_291004","x1_291586","x1_291633","x1_294241","x1_295771","x1_295995","x1_297421","x1_298473","x1_298641","x1_298702","x1_298859","x1_304165","x1_305927","x1_307894","x1_308042","x1_309011","x1_309311","x1_309606","x1_311787","x1_311893","x1_312367","x1_313420","x1_314907","x1_315131","x1_315770","x1_317403","x1_319131","x1_319269","x1_319438","x1_319720","x1_319839","x1_319870","x1_320006","x1_320308","x1_321198","x1_324449","x1_325052","x1_325208","x1_325468","x1_327057","x1_328284","x1_328438","x1_328955","x1_329022","x1_330740","x1_331693","x1_335679","x1_335820","x1_336211","x1_336243","x1_337611","x1_337641","x1_339309","x1_340366","x1_340457","x1_340564","x1_341084","x1_341163","x1_341577","x1_341947","x1_343355","x1_343756","x1_344229","x1_344391","x1_344960","x1_345120","x1_345280","x1_345561","x1_345637","x1_347304","x1_348168","x1_348432","x1_348514","x1_348914","x1_348958","x1_349453","x1_350645","x1_350949","x1_351055","x1_352942","x1_353188","x1_353320","x1_353555","x1_353719","x1_354277","x1_354458","x1_354566","x1_355003","x1_355207","x1_355523","x1_356109","x1_356288","x1_356293","x1_356579","x1_356814","x1_356854","x1_357296","x1_357354","x1_358099","x1_358224","x1_358227","x1_358744","x1_359062","x1_359250","x1_359344","x1_359530","x1_360394","x1_360447","x1_361476","x1_361752","x1_362388","x1_362748","x1_363263","x1_363325","x1_363573","x1_364349","x1_364537","x1_364894","x1_367031","x1_368316","x1_368352","x1_369210","x1_369916","x1_370457","x1_371350","x1_373314","x1_373375","x1_374326","x1_374839","x1_375388","x1_375457","x1_375887","x1_376747","x1_376850","x1_377073","x1_377560","x1_377689","x1_377727","x1_378280","x1_379326","x1_379399","x1_379926","x1_380209","x1_380925","x1_381702","x1_381805","x1_381885","x1_381935","x1_382147","x1_382240","x1_382723","x1_382983","x1_383964","x1_384931","x1_385874","x1_387659","x1_388175","x1_389467","x1_390327","x1_390609","x1_392139","x1_392377","x1_392757","x1_393163","x1_393276","x1_394279","x1_394299","x1_394390","x1_394844","x1_395079","x1_395897","x1_397512","x1_397957","x1_399329","x1_399994","x1_400354","x1_400648","x1_400742","x1_402560","x1_402649","x1_404264","x1_405618","x1_407074","x1_407718","x1_408086","x1_408190","x1_408341","x1_408558","x1_408854","x1_408904","x1_412420","x1_412633","x1_413100","x1_414441","x1_416539","x1_423207","x1_424600","x1_426415","x1_427372","x1_427538","x1_428081","x1_430639","x1_430754","x1_431684","x1_432033","x1_432102","x1_432816","x1_433410","x1_433829","x1_441135","x1_441914","x1_444053","x1_444920","x1_445199","x1_446649","x1_448681","x1_449692","x1_450834","x1_451858","x1_452258","x1_452493","x1_454359","x1_454726","x1_455271","x1_455300","x1_461046","x1_465506","x1_469295","x1_470370","x1_471442","x1_473303","x1_474836","x1_477335","x1_477788","x1_480047","x1_487971","x1_499579","x1_500718","x1_503371","x1_505257","x1_517565","x1_521535","x1_521665","x1_525640","x1_526179","x1_526894","x1_527440","x1_528288","x1_528870","x1_533137","x1_533212","x1_535596","x1_536511","x1_537265","x1_537784","x1_540803"],axis=1)
t_test=t_test.drop(["x1_267147","x1_267226","x1_268233","x1_268288","x1_269638","x1_271639","x1_271661","x1_271814","x1_272055","x1_272245","x1_272727","x1_273450","x1_273888","x1_275721","x1_275964","x1_277007","x1_277318","x1_278050","x1_278306","x1_278554","x1_279317","x1_279350","x1_279871","x1_280144","x1_281773","x1_282968","x1_283346","x1_285154","x1_285622","x1_287956","x1_288450","x1_291004","x1_291586","x1_291633","x1_294241","x1_295771","x1_295995","x1_297421","x1_298473","x1_298641","x1_298702","x1_298859","x1_304165","x1_305927","x1_307894","x1_308042","x1_309011","x1_309311","x1_309606","x1_311787","x1_311893","x1_312367","x1_313420","x1_314907","x1_315131","x1_315770","x1_317403","x1_319131","x1_319269","x1_319438","x1_319720","x1_319839","x1_319870","x1_320006","x1_320308","x1_321198","x1_324449","x1_325052","x1_325208","x1_325468","x1_327057","x1_328284","x1_328438","x1_328955","x1_329022","x1_330740","x1_331693","x1_335679","x1_335820","x1_336211","x1_336243","x1_337611","x1_337641","x1_339309","x1_340366","x1_340457","x1_340564","x1_341084","x1_341163","x1_341577","x1_341947","x1_343355","x1_343756","x1_344229","x1_344391","x1_344960","x1_345120","x1_345280","x1_345561","x1_345637","x1_347304","x1_348168","x1_348432","x1_348514","x1_348914","x1_348958","x1_349453","x1_350645","x1_350949","x1_351055","x1_352942","x1_353188","x1_353320","x1_353555","x1_353719","x1_354277","x1_354458","x1_354566","x1_355003","x1_355207","x1_355523","x1_356109","x1_356288","x1_356293","x1_356579","x1_356814","x1_356854","x1_357296","x1_357354","x1_358099","x1_358224","x1_358227","x1_358744","x1_359062","x1_359250","x1_359344","x1_359530","x1_360394","x1_360447","x1_361476","x1_361752","x1_362388","x1_362748","x1_363263","x1_363325","x1_363573","x1_364349","x1_364537","x1_364894","x1_367031","x1_368316","x1_368352","x1_369210","x1_369916","x1_370457","x1_371350","x1_373314","x1_373375","x1_374326","x1_374839","x1_375388","x1_375457","x1_375887","x1_376747","x1_376850","x1_377073","x1_377560","x1_377689","x1_377727","x1_378280","x1_379326","x1_379399","x1_379926","x1_380209","x1_380925","x1_381702","x1_381805","x1_381885","x1_381935","x1_382147","x1_382240","x1_382723","x1_382983","x1_383964","x1_384931","x1_385874","x1_387659","x1_388175","x1_389467","x1_390327","x1_390609","x1_392139","x1_392377","x1_392757","x1_393163","x1_393276","x1_394279","x1_394299","x1_394390","x1_394844","x1_395079","x1_395897","x1_397512","x1_397957","x1_399329","x1_399994","x1_400354","x1_400648","x1_400742","x1_402560","x1_402649","x1_404264","x1_405618","x1_407074","x1_407718","x1_408086","x1_408190","x1_408341","x1_408558","x1_408854","x1_408904","x1_412420","x1_412633","x1_413100","x1_414441","x1_416539","x1_423207","x1_424600","x1_426415","x1_427372","x1_427538","x1_428081","x1_430639","x1_430754","x1_431684","x1_432033","x1_432102","x1_432816","x1_433410","x1_433829","x1_441135","x1_441914","x1_444053","x1_444920","x1_445199","x1_446649","x1_448681","x1_449692","x1_450834","x1_451858","x1_452258","x1_452493","x1_454359","x1_454726","x1_455271","x1_455300","x1_461046","x1_465506","x1_469295","x1_470370","x1_471442","x1_473303","x1_474836","x1_477335","x1_477788","x1_480047","x1_487971","x1_499579","x1_500718","x1_503371","x1_505257","x1_517565","x1_521535","x1_521665","x1_525640","x1_526179","x1_526894","x1_527440","x1_528288","x1_528870","x1_533137","x1_533212","x1_535596","x1_536511","x1_537265","x1_537784","x1_540803"],axis=1)

##### x_sample['B']  = np.where(x_sample['B'] == 'a', 'aa', x_sample['B'])

##### ohe = OneHotEncoder()
train_latitude = ohe.fit_transform(x[['x1']])

##### s_locate = pd.DataFrame(train_latitude, columns=[col for col in ohe.categories_[0]])

##### for i in df_ho['locdate']:
    x.loc[(x['base_date'] == i),'base_date'] = 1
x.loc[(x['base_date'] > 1),'base_date'] = 0

for i in df_ho['locdate']:
    t_test.loc[(t_test['base_date'] == i),'base_date'] = 1
    
t_test.loc[(t_test['base_date'] > 1),'base_date'] = 0    
x = x.replace('월', 0)
x = x.replace('화', 0)
x = x.replace('수', 0)
x = x.replace('목', 0)
x = x.replace('금', 0)
x = x.replace('토', 1)
x = x.replace('일', 1)
t_test = t_test.replace('월', 0)
t_test = t_test.replace('화', 0)
t_test = t_test.replace('수', 0)
t_test = t_test.replace('목', 0)
t_test = t_test.replace('금', 0)
t_test = t_test.replace('토', 1)
t_test = t_test.replace('일', 1)

##### str_list = ['start_latitude','start_longitude','end_latitude','end_longitude']
for i in str_list:
    if i == 'start_latitude' or i == 'end_latitude':
        X_train[i] -= 33
    else:
        X_train[i] -= 126
for i in str_list:
    if i == 'start_latitude' or i == 'end_latitude':
        test[i] -= 33
    else:
        test[i] -= 126

##### xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', max_depth = 5)
xg_reg.fit(x, y)
pred = xg_reg.predict(test)


In [42]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_test, y, test_size=0.2, random_state=42)

In [43]:
## 훈련데이터세트의 shape ##
print(x.shape)
# (569,30) 

## 훈련세트, 테스트세트 shape ##
print(x_train.shape, x_test.shape)
# (455,30) (114,30)

## 훈련데이터세트의 클래스 비율 ##
np.unique(y, return_counts = True)
# (array([0,1]), array([212,357]))

## 훈련세트의 클래스 비율 ##
np.unique(y_train, return_counts = True)
# (array([0,1]), array([170,285]))

## 훈련세트의 클래스 비율 ##
np.unique(y_test, return_counts = True)
# (array([0,1]), array([42, 72]))

(4701217, 691)
(3760973, 690) (940244, 690)


(array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
        27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
        40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52.,
        53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65.,
        66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77., 78.,
        79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89., 90., 91.,
        92., 93., 94., 99.]),
 array([   13,    37,    67,   136,   262,   485,   766,  1026,  1433,
         2231,  2645,  3737,  4243,  5134,  6256,  7376,  8889, 10208,
        11071, 11840, 13013, 13406, 14214, 14862, 15329, 15951, 16375,
        17550, 18409, 19268, 19691, 20069, 20536, 18862, 18767, 18855,
        18113, 18710, 18326, 18524, 18595, 18463, 18582, 18707, 19559,
        20115, 21465, 22336, 22063, 21496, 21551, 21307, 21547, 21719,
        20974, 18471, 17351, 1523

##### xg_reg = xgb.XGBRegressor(random_state=42).fit(x,y)

##### pred = xg_reg.predict(t_test)

##### from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(x_sample, y_sample)

##### pred = reg.predict(t_test)

##### reg.score(X, y)

In [43]:
del x

NameError: name 'x' is not defined

In [44]:
del y

In [39]:
x_sample_arr = x.to_numpy()
y_sample_arr = y.to_numpy()

##### x.to_csv("C:/Users/82108/JejuRoad/JejuRoadData/open/data_x.csv", index = False)

##### y.to_csv("C:/Users/82108/JejuRoad/JejuRoadData/open/data_y.csv", index = False)
t_test.to_csv("C:/Users/82108/JejuRoad/JejuRoadData/open/data_t_tset.csv", index = False)

##### x1 = x.iloc[:len(x)//2,:]
x2 = x.iloc[len(x)//2:,:]

In [40]:
xg_reg = xgb.XGBRegressor(max_depth=4,random_state=42).fit(x_sample_arr, y_sample_arr)

XGBoostError: bad allocation

In [93]:
xg_reg.score(x_sample, y_sample)

0.9762103480393274

In [82]:
pred = xg_reg.predict(t_test)

In [83]:
sample_submission = pd.read_csv('C:/Users/82108/JejuRoad/JejuRoadData/open/sample_submission.csv')

In [84]:
sample_submission['target'] = pred
sample_submission.to_csv("C:/Users/82108/JejuRoad/JejuRoadData/open/submit.csv", index = False)

In [85]:
sample_submission

,id,target
0,TEST_000000,47.276073
1,TEST_000001,50.422585
2,TEST_000002,48.797817
3,TEST_000003,50.356426
4,TEST_000004,34.801506
...,...,...
291236,TEST_291236,46.182945
291237,TEST_291237,34.481918
291238,TEST_291238,29.469133
291239,TEST_291239,42.706917


In [73]:
sample_submission

,id,target
0,TEST_000000,46.128448
1,TEST_000001,47.038200
2,TEST_000002,49.431301
3,TEST_000003,55.405304
4,TEST_000004,37.088409
...,...,...
291236,TEST_291236,47.865223
291237,TEST_291237,37.247200
291238,TEST_291238,30.001467
291239,TEST_291239,44.591904


In [47]:
suspect_submission_1 = sample_submission.copy()
suspect_submission_1

,id,target
0,TEST_000000,0
1,TEST_000001,0
2,TEST_000002,0
3,TEST_000003,0
4,TEST_000004,0
...,...,...
291236,TEST_291236,0
291237,TEST_291237,0
291238,TEST_291238,0
291239,TEST_291239,0


In [94]:
suspect_submission = sample_submission.copy()

In [83]:
sample_submission

,id,target
0,TEST_000000,26.916473
1,TEST_000001,46.055325
2,TEST_000002,51.405029
3,TEST_000003,41.738152
4,TEST_000004,31.058563
...,...,...
291236,TEST_291236,47.202000
291237,TEST_291237,37.745926
291238,TEST_291238,22.922579
291239,TEST_291239,39.377304


In [95]:
suspect_submission

,id,target
0,TEST_000000,24.408371
1,TEST_000001,45.870052
2,TEST_000002,63.367569
3,TEST_000003,34.505859
4,TEST_000004,37.616901
...,...,...
291236,TEST_291236,51.246002
291237,TEST_291237,52.160847
291238,TEST_291238,21.929148
291239,TEST_291239,25.345930
